## Chapter 16-18

In [7]:
!pip install pdfplumber
!pip install spacy


In [8]:
import re

In [9]:
import pdfplumber


In [6]:
## Path to the pdf
pdf_path = "/content/Kenya Constitution.pdf"

with pdfplumber.open(pdf_path) as pdf:
  for page in pdf.pages:
      text = page.extract_text()
      print(text)



Streaming output truncated to the last 5000 lines.
Article.
Constitution of Kenya, 2010 65
PART 3—OFFICES OF PARLIAMENT
Speakers and Deputy Speakers of Parliament.
106. (1) There shall be—
(a) a Speaker for each House of Parliament, who shall be elected
by that House in accordance with the Standing Orders, from
among persons who are qualified to be elected as members
of Parliament but are not such members; and
(b) a Deputy Speaker for each House of Parliament, who shall be
elected by that House in accordance with the Standing
Orders, from among the members of that House.
(2) The office of Speaker or Deputy Speaker shall become
vacant—
(a) when a new House of Parliament first meets after an election;
(b) if the office holder, as a member of the relevant House,
vacates office under Article 103;
(c) if the relevant House so resolves by resolution supported by
the votes of at least two-thirds of its members; or
(d) if the office holder resigns from office in a letter addressed to
the relev

In [10]:
## Function to extract text from specific pages

def extract_specific_pages(pdf_path, start_page, end_page):
  with pdfplumber.open(pdf_path) as pdf:
    num_pages = len(pdf.pages) # Get the total number of pages in the PDF.
    extracted_text = ""
    # Adjust page numbers to be within the valid range
    start_page = max(0, min(start_page, num_pages - 1))
    end_page = max(0, min(end_page, num_pages))
    for page_num in range(start_page, end_page):
      page = pdf.pages[page_num]
      extracted_text += page.extract_text() + "\n"
    return extracted_text

### Chapter 13

In [11]:
chapter_13 = extract_specific_pages(pdf_path, 138,143)
chapter_13

'Constitution of Kenya, 2010 139\nCHAPTER THIRTEEN—THE PUBLIC SERVICE\nPART 1—VALUES AND PRINCIPLES OF PUBLIC SERVICE\nValues and principles of public service.\n232. (1) The values and principles of public service include—\n(a) high standards of professional ethics;\n(b) efficient, effective and economic use of resources;\n(c) responsive, prompt, effective, impartial and equitable\nprovision of services;\n(d) involvement of the people in the process of policy making;\n(e) accountability for administrative acts;\n(f) transparency and provision to the public of timely, accurate\ninformation;\n(g) subject to paragraphs (h) and (i), fair competition and merit as\nthe basis of appointments and promotions;\n(h) representation of Kenya’s diverse communities; and\n(i) affording adequate and equal opportunities for appointment,\ntraining and advancement, at all levels of the public service,\nof—\n(i) men and women;\n(ii) the members of all ethnic groups; and\n(iii) persons with disabilities.\n(

Since chapter 14 is included in the last page of cahpter 13, let's trim it to only contain information on chapter 13

In [12]:
chapter_13_trimmed = chapter_13.split("CHAPTER FOURTEEN")[0].strip()
chapter_13_trimmed

'Constitution of Kenya, 2010 139\nCHAPTER THIRTEEN—THE PUBLIC SERVICE\nPART 1—VALUES AND PRINCIPLES OF PUBLIC SERVICE\nValues and principles of public service.\n232. (1) The values and principles of public service include—\n(a) high standards of professional ethics;\n(b) efficient, effective and economic use of resources;\n(c) responsive, prompt, effective, impartial and equitable\nprovision of services;\n(d) involvement of the people in the process of policy making;\n(e) accountability for administrative acts;\n(f) transparency and provision to the public of timely, accurate\ninformation;\n(g) subject to paragraphs (h) and (i), fair competition and merit as\nthe basis of appointments and promotions;\n(h) representation of Kenya’s diverse communities; and\n(i) affording adequate and equal opportunities for appointment,\ntraining and advancement, at all levels of the public service,\nof—\n(i) men and women;\n(ii) the members of all ethnic groups; and\n(iii) persons with disabilities.\n(

In [13]:
import spacy

In [14]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are Values and principles of public service?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)


value principle public service


 Now let's split the chapter into three different sections which will increase the chatbot's accuracy in answering questions.

In [15]:
def split_chapter(chapter_text):
  #split at key headings and split extra white spaces

  sections = {
      "Values and principles of public service":[],
      "The Public Service Commission":[],
      "Functions and powers of the Public Service Commission":[]
  }

  ## Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None

  for line in lines:
    line = line.strip()
    # Use line instead of stripped_line
    if line.startswith("Values and principles of public service"):
      current_section = "Values and principles of public service"
    elif line.startswith("The Public Service Commission"):
        current_section = "The Public Service Commission"
    elif line.startswith("Functions and powers of the Public Service Commission"):
        current_section = "Functions and powers of the Public Service Commission"
        ## Append line to the current section if it's set
    if current_section:
        sections[current_section].append(line) # Use line instead of stripped_line
  ## Join each section into single string
  for key in sections:
    sections[key] = "\n".join(sections[key])
  return sections

In [16]:
 ## split the chapter into sections
chapter_13_sections = split_chapter(chapter_13_trimmed)
chapter_13_sections

print("Values and principles of public service Section:\n", chapter_13_sections["Values and principles of public service"])
print("The Public Service Commission Section:\n", chapter_13_sections["The Public Service Commission"])
print("Functions and powers of the Public Servic Commision Section:\n", chapter_13_sections['Functions and powers of the Public Service Commission'])

Values and principles of public service Section:
 Values and principles of public service.
232. (1) The values and principles of public service include—
(a) high standards of professional ethics;
(b) efficient, effective and economic use of resources;
(c) responsive, prompt, effective, impartial and equitable
provision of services;
(d) involvement of the people in the process of policy making;
(e) accountability for administrative acts;
(f) transparency and provision to the public of timely, accurate
information;
(g) subject to paragraphs (h) and (i), fair competition and merit as
the basis of appointments and promotions;
(h) representation of Kenya’s diverse communities; and
(i) affording adequate and equal opportunities for appointment,
training and advancement, at all levels of the public service,
of—
(i) men and women;
(ii) the members of all ethnic groups; and
(iii) persons with disabilities.
(2) The values and principles of public service apply to public
service in—
(a) all State

In [17]:
chapter_13_sections

{'Values and principles of public service': 'Values and principles of public service.\n232. (1) The values and principles of public service include—\n(a) high standards of professional ethics;\n(b) efficient, effective and economic use of resources;\n(c) responsive, prompt, effective, impartial and equitable\nprovision of services;\n(d) involvement of the people in the process of policy making;\n(e) accountability for administrative acts;\n(f) transparency and provision to the public of timely, accurate\ninformation;\n(g) subject to paragraphs (h) and (i), fair competition and merit as\nthe basis of appointments and promotions;\n(h) representation of Kenya’s diverse communities; and\n(i) affording adequate and equal opportunities for appointment,\ntraining and advancement, at all levels of the public service,\nof—\n(i) men and women;\n(ii) the members of all ethnic groups; and\n(iii) persons with disabilities.\n(2) The values and principles of public service apply to public\nservice in

Question and Answer mapping based on key phrases and corresponding sections.

In [18]:
## Define the sections variable with Chapter 13
sections = chapter_13_sections

#Define the Q&A mapping
QA_mapping = {
    "Values": "Values and principles of public service",
    "Public Service Commission": "The Public Service Commission",
    "Functions": "Functions and powers of the Public Service Commission"
}

##define the preprocess query function
def preprocess_query(query):
  return query

##Update the Q&A system to use preprocessed queries
def answer_question_nlp(query):

  #Preprocess the query
  preprocessed_query = preprocess_query(query)

  #Search for a key in QA_mapping that matches the preprocessed query
  for key, value in QA_mapping.items(): # Iterate through key-value pairs of QA_mapping
    if key in preprocessed_query:
    ## Return the relevant section in the text
      return sections[value] # Access sections using the full section name (value in QA_mapping)
  return "Sorry, I couldn't find an answer to your question."

##Example
user_query = "What are Functions of the public service?"
answer = answer_question_nlp(user_query) # Pass the user_query
print(answer)

Functions and powers of the Public Service Commission.
234. (1) The functions and powers of the Commission are as set
out in this Article.
(2) The Commission shall—
(a) subject to this Constitution and legislation—
(i) establish and abolish offices in the public service; and
(ii) appoint persons to hold or act in those offices, and to
confirm appointments;
Constitution of Kenya, 2010 141
(b) exercise disciplinary control over and remove persons holding
or acting in those offices;
(c) promote the values and principles referred to in Articles 10
and 232 throughout the public service;
(d) investigate, monitor and evaluate the organisation,
administration and personnel practices of the public service;
(e) ensure that the public service is efficient and effective;
(f) develop human resources in the public service;
(g) review and make recommendations to the national
government in respect of conditions of service, code of
conduct and qualifications of officers in the public service;
(h) evalu

### Chapter 14


In [19]:
chapter_14 = extract_specific_pages(pdf_path, 142,150)
chapter_14

'Constitution of Kenya, 2010 143\n(c) to assign teachers employed by the Commission for service in\nany public school or institution;\n(d) to promote and transfer teachers;\n(e) to exercise disciplinary control over teachers; and\n(f) to terminate the employment of teachers.\n(3) The Commission shall—\n(a) review the standards of education and training of persons\nentering the teaching service;\n(b) review the demand for and the supply of teachers; and\n(c) advise the national government on matters relating to the\nteaching profession.\nCHAPTER FOURTEEN—NATIONAL SECURITY\nPART 1—NATIONAL SECURITY ORGANS\nPrinciples of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following princi

Since Chapter 14 starts in the same page as the end of chapter 13.Let's trim it.


In [20]:
# Trim the chapter_14 text to start from "CHAPTER FOURTEEN"
chapter_14_trimmed = chapter_14[chapter_14.find("CHAPTER FOURTEEN"):].strip()

## Further trim to remove Chapter 15 if it's present on the same page
if "CHAPTER FIFTEEN" in chapter_14_trimmed:
  chapter_14_trimmed = chapter_14_trimmed.split("CHAPTER FIFTEEN")[0].strip()


chapter_14_trimmed

'CHAPTER FOURTEEN—NATIONAL SECURITY\nPART 1—NATIONAL SECURITY ORGANS\nPrinciples of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable propo

In [21]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are the principles of national security?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)


principle national security


Let's split the chapter into sections.


In [22]:
def split_chapter_14(chapter_text):
  #split at key headings and split extra white spaces

  sections = {
      "Principles of National Security":[],
      "National security organs":[],
      "Establishment of the National Security Council":[],
      "Establishmnet of Defence Forces and Defence Council":[],
      "Establishment of National Intelligence Service":[],
      "Establishment of the National Police Service":[],
      "Objects and Functions the National Police Service":[],
      "Command of the National Police Service":[],
      "National Police Service Commission":[],
      "Other police services":[]
  }

  ## Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None
  for line in lines:
    line = line.strip()
    if "Principles of National Security" in line:
        current_section = "Principles of National Security"
    elif "National security organs" in line:
        current_section = "National security organs"
    elif "Establishment of the National Security Council" in line:
        current_section = "Establishment of the National Security Council"
    elif "Establishmnet of Defence Forces and Defence Council" in line:
        current_section = "Establishmnet of Defence Forces and Defence Council"
    elif "Establishment of National Intelligence Service" in line:
        current_section = "Establishment of National Intelligence Service"
    elif "Establishment of the National Police Service" in line:
        current_section = "Establishment of the National Police Service"
    elif "Objects and Functions the National Police Service" in line:
        current_section = "Objects and Functions the National Police Service"
    elif "Command of the National Police Service" in line:
        current_section = "Command of the National Police Service"
    elif "National Police Service Commission" in line:
        current_section = "National Police Service Commission"
    elif "Other police services" in line:
        current_section = "Other police services"
        ## Append line to the current section if it's set
    if current_section:
        sections[current_section].append(line) # Use line instead of stripped_line
  ## Join each section into single string
  for key in sections:
    sections[key] = "\n".join(sections[key])
  return sections


chapter_14_sections = split_chapter_14(chapter_14_trimmed)


In [23]:
print("Principles of National Security Section:\n", chapter_14_sections["Principles of National Security"])
print("National security organs Section:\n", chapter_14_sections["National security organs"])

Principles of National Security Section:
 
National security organs Section:
 National security organs.
239. (1) The national security organs are—
144 Constitution of Kenya, 2010
(a) the Kenya Defence Forces;
(b) the National Intelligence Service; and
(c) the National Police Service.
(2) The primary object of the national security organs and
security system is to promote and guarantee national security in
accordance with the principles mentioned in Article 238 (2).
(3) In performing their functions and exercising their powers, the
national security organs and every member of the national security
organs shall not—
(a) act in a partisan manner;
(b) further any interest of a political party or cause; or
(c) prejudice a political interest or political cause that is legitimate
under this Constitution.
(4) A person shall not establish a military, paramilitary, or similar
organisation that purports to promote and guarantee national security,
except as provided for by this Constitution or an 

In [24]:
chapter_14_sections

{'Principles of National Security': '',
 'National security organs': 'National security organs.\n239. (1) The national security organs are—\n144 Constitution of Kenya, 2010\n(a) the Kenya Defence Forces;\n(b) the National Intelligence Service; and\n(c) the National Police Service.\n(2) The primary object of the national security organs and\nsecurity system is to promote and guarantee national security in\naccordance with the principles mentioned in Article 238 (2).\n(3) In performing their functions and exercising their powers, the\nnational security organs and every member of the national security\norgans shall not—\n(a) act in a partisan manner;\n(b) further any interest of a political party or cause; or\n(c) prejudice a political interest or political cause that is legitimate\nunder this Constitution.\n(4) A person shall not establish a military, paramilitary, or similar\norganisation that purports to promote and guarantee national security,\nexcept as provided for by this Constitut

Question and Answer mapping for Chapter 14


In [25]:
##Define Q&A mapping
QA_mapping = {
    "Principles of National Security": "Principles of National Security",
    "National security organs": "National security organs",
    "National Security Council": "Establishment of the National Security Council",
    "Defence Forces": "Establishmnet of Defence Forces and Defence Council",
    "National Intelligence Service": "Establishment of National Intelligence Service",
    "National Police Service": "Establishment of the National Police Service",
    "Objects and Functions of the National Police Service": "Objects and Functions the National Police Service",
    "Command of the National Police Service": "Command of the National Police Service",
    "National Police Service Commission": "National Police Service Commission",
    "Other police services": "Other police services"
}

##define the preprocess query function
def preprocess_query(query):
  return query

##Update the Q&A system to use preprocessed queries
def answer_question_nlp_chapter_14(query):

  #Preprocess the query
  preprocessed_query = preprocess_query(query)

  #Search for a key in QA_mapping that matches the preprocessed query
  for key, value in QA_mapping.items(): # Iterate through key-value pairs of QA_mapping
    if key.lower() in preprocessed_query.lower():
    ## Return the relevant section in the text
      return chapter_14_sections[value] # Access sections using the full section name (value in QA_mapping)
  return "Sorry, I couldn't find an answer to your question."

##Example
user_query = "What are the National Security Organs?"
answer = answer_question_nlp_chapter_14(user_query) # Pass the user_query
answer

'National security organs.\n239. (1) The national security organs are—\n144 Constitution of Kenya, 2010\n(a) the Kenya Defence Forces;\n(b) the National Intelligence Service; and\n(c) the National Police Service.\n(2) The primary object of the national security organs and\nsecurity system is to promote and guarantee national security in\naccordance with the principles mentioned in Article 238 (2).\n(3) In performing their functions and exercising their powers, the\nnational security organs and every member of the national security\norgans shall not—\n(a) act in a partisan manner;\n(b) further any interest of a political party or cause; or\n(c) prejudice a political interest or political cause that is legitimate\nunder this Constitution.\n(4) A person shall not establish a military, paramilitary, or similar\norganisation that purports to promote and guarantee national security,\nexcept as provided for by this Constitution or an Act of Parliament.\n(5) The national security organs are su

### Chapter 14 done.

### Now Chapter 15

In [26]:
chapter_15 = extract_specific_pages(pdf_path, 148,155)
chapter_15

'Constitution of Kenya, 2010 149\nCommission.\n(2) The Commission consists of—\n(a) the following persons, each appointed by the President—\n(i) a person who is qualified to be appointed as a High\nCourt Judge;\n(ii) two retired senior police officers; and\n(iii) three persons of integrity who have served the public with\ndistinction;\n(b) the Inspector-General of the National Police Service; and\n(c) both Deputy Inspectors-General of the National Police\nService.\n(3) The Commission shall—\n(a) recruit and appoint persons to hold or act in offices in the\nservice, confirm appointments, and determine promotions\nand transfers within the National Police Service;\n(b) observing due process, exercise disciplinary control over and\nremove persons holding or acting in offices within the\nService; and\n(c) perform any other functions prescribed by national legislation.\n(4) The composition of the National Police Service shall reflect\nthe regional and ethnic diversity of the people of Kenya.

In [27]:
# Trim the chapter_15 text to start from "CHAPTER FIFTEEN"
chapter_15_trimmed = chapter_15[chapter_15.find("CHAPTER FIFTEEN"):].strip()

# Further trim to remove Chapter 16 if it's present on the same page
if "CHAPTER SIXTEEN" in chapter_15_trimmed:
  chapter_15_trimmed = chapter_15_trimmed.split("CHAPTER SIXTEEN")[0].strip()

chapter_15_trimmed

'CHAPTER FIFTEEN—COMMISSIONS AND INDEPENDENT\nOFFICES\nApplication of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the ind

In [28]:
##Further trim to remove Chapter 16 if it's present on the same page

if "CHAPTER SIXTEEN" in chapter_15_trimmed:
  chapter_15_trimmed = chapter_15_trimmed.split("CHAPTER SIXTEEN")[0].strip()

chapter_15_trimmed

'CHAPTER FIFTEEN—COMMISSIONS AND INDEPENDENT\nOFFICES\nApplication of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the ind

In [29]:
## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are the commissions and independent offices?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)

commission independent office


Now let's split chapter fifteen into sections.

In [36]:
def split_chapter_15(chapter_text):
  #split at key headings and split extra white spaces
  # Initialize sections as a dictionary with list values
  sections = {
      "Application of Chapter": [],
      "Objects,authority and funding of commissions and independent offices": [],
      "Composition, appointment and terms of office": [],
      "Removal from office": [],
      "General functions and powers": [],
      "Incorporation of commissions and independent offices": [],
      "Reporting by commissions and independent offices": []
  }

  ## Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None

  for line in lines:
      line = line.strip()
      if line.startswith("Application of Chapter"):
          current_section = "Application of Chapter"
      elif line.startswith("Objects,authority and funding of commissions and independent offices"):
          current_section = "Objects,authority and funding of commissions and independent offices"
      elif line.startswith("Composition, appointment and terms of office"):
          current_section = "Composition, appointment and terms of office"
      elif line.startswith("Removal from office"):
          current_section = "Removal from office"
      elif line.startswith("General functions and powers"):
          current_section = "General functions and powers"
      elif line.startswith("Incorporation of commissions and independent offices"):
          current_section = "Incorporation of commissions and independent offices"
      elif line.startswith("Reporting by commissions and independent offices"):
          current_section = "Reporting by commissions and independent offices"
          ## Append line to the current section if it's set
      if current_section and line:  # Check if current_section is set and line is not empty
          sections[current_section].append(line)
  ## Join each section into single string
  for key in sections:
        sections[key] = "\n".join(sections[key])
  return sections

chapter_15_sections = split_chapter_15(chapter_15_trimmed)
 ## Example
print("Application of Chapter Section:\n", chapter_15_sections["Application of Chapter"])
print("Objects,authority and funding of commissions and independent offices Section:\n", chapter_15_sections["Objects,authority and funding of commissions and independent offices"])

Application of Chapter Section:
 Application of Chapter.
248. (1) This Chapter applies to the commissions specified in
clause (2) and the independent offices specified in clause (3), except
to the extent that this Constitution provides otherwise.
(2) The commissions are—
150 Constitution of Kenya, 2010
(a) the Kenya National Human Rights and Equality Commission;
(b) the National Land Commission;
(c) the Independent Electoral and Boundaries Commission;
(d) the Parliamentary Service Commission;
(e) the Judicial Service Commission;
(f) the Commission on Revenue Allocation;
(g) the Public Service Commission;
(h) the Salaries and Remuneration Commission;
(i) the Teachers Service Commission; and
(j) the National Police Service Commission.
(3) The independent offices are—
(a) the Auditor-General; and
(b) the Controller of Budget.
Objects, authority and funding of commissions and independent
offices.
249. (1) The objects of the commissions and the independent
offices are to—
(a) protect the so

In [37]:
chapter_15_sections

{'Application of Chapter': 'Application of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the independent\noffices are to—\n

### Question and Answer mapping based on key phrases and corresponding section in chapter 15

In [39]:
##Define Q&A mapping for Chapter 15
QA_mapping_chapter_15 = {
    "Application of Chapter": "Application of Chapter",
    "Objects,authority and funding of commissions and independent offices": "Objects,authority and funding of commissions and independent offices",
    "Composition, appointment and terms of office": "Composition, appointment and terms of office",
    "Removal from office": "Removal from office",
    "General functions and powers": "General functions and powers",
    "Incorporation of commissions and independent offices": "Incorporation of commissions and independent offices",
    "Reporting by commissions and independent offices": "Reporting by commissions and independent offices"
}


##define the preprocess query function
def preprocess_query(query):
  return query


##Update the Q&A system to use preprocessed queries
def answer_question_nlp_chapter_15(query):

  #Preprocess the query
  preprocessed_query = preprocess_query(query)

  #Search for a key in QA_mapping that matches the preprocessed query
  for key, value in QA_mapping_chapter_15.items(): # Iterate through key-value pairs of QA_mapping
    if key.lower() in preprocessed_query.lower():
    ## Return the relevant section in the text
      return chapter_15_sections[value] # Access sections using the full section name (value in QA_mapping)
  return "Sorry, I couldn't find an answer to your question."

##Example
user_query = "What are the general functions and powers of commissions and independent offices?"
answer = answer_question_nlp_chapter_15(user_query) # Pass the user_query
answer



'General functions and powers.\n252. (1) Each commission, and each holder of an independent\noffice—\n(a) may conduct investigations on its own initiative or on a\ncomplaint made by a member of the public;\n(b) has the powers necessary for conciliation, mediation and\nnegotiation;\n(c) shall recruit its own staff; and\n(d) may perform any functions and exercise any powers\nprescribed by legislation, in addition to the functions and\npowers conferred by this Constitution.\n(2) A complaint to a commission or the holder of an independent\noffice may be made by any person entitled to institute court\nproceedings under Article 22 (1) and (2).\n(3) The following commissions and independent offices have the\npower to issue a summons to a witness to assist for the purposes of its\ninvestigations—\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the Judicial Service Commission;\n154 Constitution of Kenya, 2010\n(c) the National Land Commission; and\n(d) the Auditor-General.'